## Assign Label Analysis

Analysis in support of creation of labelling strategy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import re

import csv
import datetime
import pickle
import os

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

from random import randint

In [2]:
# Functions shared across multiple notebooks are stored in yahoo_data_ext_kit.py which can be found in the same
# directory as the notebooks.

import yahoo_data_ext_kit as ext

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
pd.set_option('display.max_rows', 900)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.0f}'.format

In [5]:
DATA_ROOT_DIR='/mnt/data/projects/MD3'
PROJ_ROOT_DIR='/home/priyesh/projects/MD3'

In [6]:
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_income_sheets_final_stage6.pkl')
income_sheets = pd.read_pickle(filepath)
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_cash_sheets_final_stage6.pkl')
cash_sheets = pd.read_pickle(filepath)
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_balance_sheets_final_stage6.pkl')
balance_sheets = pd.read_pickle(filepath)

In [7]:
income_sheets = income_sheets.fillna(0)
cash_sheets = cash_sheets.fillna(0)
balance_sheets = balance_sheets.fillna(0)

## Balance Sheets

In [8]:
balance_sheets['totalDebt'] = balance_sheets['currentDebt'] + balance_sheets['longTermDebt']
balance_sheets['totalAssets'] = balance_sheets['currentAssets'] + balance_sheets['totalNonCurrentAssets']
balance_sheets['totalLiabilities'] = balance_sheets['currentLiabilities'] + balance_sheets['totalNonCurrentLiabilities']
balance_sheets['workingCapital'] = balance_sheets['currentAssets'] - balance_sheets['currentLiabilities']

In [9]:
# Calculate Ratios 

balance_sheets['quickRatio'] = (balance_sheets['currentAssets'] - balance_sheets['inventory']) / balance_sheets['currentLiabilities']
balance_sheets['currentRatio'] = balance_sheets['currentAssets'] / balance_sheets['currentLiabilities']
balance_sheets['debtCapitalRatio'] = 100 * balance_sheets['totalDebt'] / (balance_sheets['totalDebt'] + balance_sheets['stockholdersEquity'])
balance_sheets['debtAssetRatio'] = 100 * balance_sheets['totalDebt'] / balance_sheets['totalAssets']
balance_sheets['FinancialLeverageRatio'] = 100 * balance_sheets['totalAssets'] / balance_sheets['stockholdersEquity']


In [10]:
list(balance_sheets.columns)

['company',
 'yahoo_sector',
 'gics_sector',
 'industry',
 'st_date',
 'st_YR',
 'st_Mnth',
 'cashEquivalent',
 'receivables',
 'finishedGoods',
 'workInProcess',
 'rawMaterials',
 'otherCurrentAssets',
 'inventory',
 'currentAssets',
 'netPPE',
 'otherNonCurrentAssets',
 'financialAssets',
 'goodwill',
 'goodwillAndOtherIntangibleAssets',
 'otherIntangibleAssets',
 'nonCurrentAccountsReceivable',
 'totalNonCurrentAssets',
 'currentDebt',
 'payablesAndAccruedExpenses',
 'otherCurrentLiabilities',
 'currentLiabilities',
 'longTermDebt',
 'otherNonCurrentLiabilities',
 'nonCurrentDeferredLiabilities',
 'nonCurrentDeferredTaxesLiabilities',
 'longTermProvisions',
 'totalNonCurrentLiabilities',
 'retainedEarnings',
 'stockholdersEquity',
 'totalDebt',
 'totalAssets',
 'totalLiabilities',
 'workingCapital',
 'quickRatio',
 'currentRatio',
 'debtCapitalRatio',
 'debtAssetRatio',
 'FinancialLeverageRatio']

In [14]:
cols=['cashEquivalent',
      'receivables',
      'finishedGoods',
      'workInProcess',
      'rawMaterials',
      'otherCurrentAssets',
      'inventory',
      'currentAssets',
      'netPPE',
      'otherNonCurrentAssets',
      'financialAssets',
      'goodwill',
      'goodwillAndOtherIntangibleAssets',
      'otherIntangibleAssets',
      'nonCurrentAccountsReceivable',
      'totalNonCurrentAssets',
      'currentDebt',
      'payablesAndAccruedExpenses',
      'otherCurrentLiabilities',
      'currentLiabilities',
      'longTermDebt',
      'otherNonCurrentLiabilities',
      'nonCurrentDeferredLiabilities',
      'nonCurrentDeferredTaxesLiabilities',
      'longTermProvisions',
      'totalNonCurrentLiabilities',
      'retainedEarnings',
      'stockholdersEquity',
      'totalDebt',
      'totalAssets',
      'totalLiabilities',
      'workingCapital',
      'quickRatio',
      'currentRatio',
      'debtCapitalRatio',
      'debtAssetRatio',
      'FinancialLeverageRatio']

df_company_bal_median = balance_sheets.groupby(['company','industry'])[cols].median().reset_index()


In [15]:
cols=[ 'quickRatio',
      'currentRatio',
      'debtCapitalRatio',
      'debtAssetRatio',
      'FinancialLeverageRatio']

df_industry_bal_median = df_company_bal_median.groupby('industry')[cols].median().reset_index()

In [22]:
# Convert to a dictionaries for easy lookup

company_bal_median = df_company_bal_median.set_index('company').T.to_dict()
industry_bal_median = df_industry_bal_median.set_index('industry').T.to_dict()

## Income Sheets

In [ ]:
list(income_sheets.columns)

In [28]:
income_sheets

,company,yahoo_sector,gics_sector,industry,st_date,st_YR,st_Mnth,totalRevenue,costOfRevenue,grossProfit,netIncome,operatingIncome,netIncomeContinuousOperations,netInterestIncome,otherIncomeExpense,operatingExpense,totalExpenses,taxProvision,SGA,researchAndDevelopment,ebit,interestExpense,interestIncome,netInterestIncome,dilutedEPS,basicEPS
3831,A,Healthcare,Health Care,Diagnostics & Research,2019-10-31,2019,10,5163000000,2358000000,2805000000,1071000000,941000000,1071000000,-38000000,16000000,1864000000,4222000000,-152000000,1460000000,404000000,993000000,74000000,36000000,-38000000,3,3
3832,A,Healthcare,Health Care,Diagnostics & Research,2020-10-31,2020,10,5339000000,2502000000,2837000000,719000000,846000000,719000000,-70000000,66000000,1991000000,4493000000,123000000,1496000000,495000000,920000000,78000000,8000000,-70000000,2,2
3833,A,Healthcare,Health Care,Diagnostics & Research,2021-10-31,2021,10,6319000000,2912000000,3407000000,1210000000,1347000000,1210000000,-79000000,92000000,2060000000,4972000000,150000000,1619000000,441000000,1441000000,81000000,2000000,-79000000,4,4
3834,A,Healthcare,Health Care,Diagnostics & Research,2022-10-31,2022,10,6848000000,3126000000,3722000000,1254000000,1618000000,1254000000,-75000000,-39000000,2104000000,5230000000,250000000,1637000000,467000000,1588000000,84000000,9000000,-75000000,4,4
945,AA,Basic Materials,Materials,Aluminum,2019-12-31,2019,12,10433000000,8537000000,1896000000,-1125000000,876000000,-853000000,-121000000,-1193000000,1020000000,9557000000,415000000,280000000,27000000,-317000000,121000000,0,-121000000,-6,-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,ZS,Technology,Information Technology,Software—Infrastructure,2022-07-31,2022,07,1090946000,242282000,848664000,-390278000,-327429000,-390278000,-51993000,-4208000,1176093000,1418375000,6648000,886954000,289139000,-327051000,56579000,4586000,-51993000,-3,-3
4406,ZTS,Healthcare,Health Care,Drug Manufacturers—Specialty & Generic,2019-12-31,2019,12,6260000000,1992000000,4268000000,1500000000,2018000000,1500000000,-223000000,6000000,2250000000,4242000000,301000000,1638000000,457000000,2024000000,223000000,37000000,-223000000,3,3
4407,ZTS,Healthcare,Health Care,Drug Manufacturers—Specialty & Generic,2020-12-31,2020,12,6675000000,2057000000,4618000000,1638000000,2269000000,1636000000,-219000000,-54000000,2349000000,4406000000,360000000,1726000000,463000000,2227000000,231000000,12000000,-219000000,3,3
4408,ZTS,Healthcare,Health Care,Drug Manufacturers—Specialty & Generic,2021-12-31,2021,12,7776000000,2303000000,5473000000,2037000000,2803000000,2034000000,-224000000,-91000000,2670000000,4973000000,454000000,2001000000,508000000,2712000000,224000000,6000000,-224000000,4,4


In [29]:
cols=['totalRevenue',
      'costOfRevenue',
      'grossProfit',
      'netIncome',
      'operatingIncome',
      'netIncomeContinuousOperations',
      'netInterestIncome',
      'otherIncomeExpense',
      'operatingExpense',
      'totalExpenses',
      'taxProvision',
      'SGA',
      'researchAndDevelopment',
      'ebit',
      'interestExpense',
      'interestIncome',
      'netInterestIncome',
      'dilutedEPS',
      'basicEPS']

df_company_inc_median = income_sheets.groupby(['company','industry'])[cols].median().reset_index()

In [30]:
# Convert to a dictionaries for easy lookup

company_inc_median = df_company_inc_median.set_index('company').T.to_dict()

## Cash Sheets

In [ ]:
list(cash_sheets.columns)

In [31]:
cols = ['netIncome',
        'netIncomeFromContinuingOperations',
        'depreciation',
        'stockBasedCompensation',
        'cashFlowOperatingActivities',
        'capitalExpenditure',
        'saleOfBusiness',
        'purchaseOfBusiness',
        'netBusinessPurchaseAndSale',
        'saleOfPPE',
        'purchaseOfPPE',
        'netPPEPurchaseAndSale',
        'saleOfInvestment',
        'purchaseOfInvestment',
        'netInvestmentPurchaseAndSale',
        'saleOfInvestmentProperties',
        'purchaseOfInvestmentProperties',
        'netInvestmentPropertiesPurchaseAndSale',
        'saleOfIntangibles',
        'purchaseOfIntangibles',
        'netIntangiblesPurchaseAndSale',
        'netOtherInvestingChanges',
        'cashFlowInvestingActivities',
        'netIssuancePaymentsOfDebt',
        'netLongTermDebtIssuance',
        'netShortTermDebtIssuance',
        'commonStockDividendPaid',
        'preferredStockDividendPaid',
        'cashDividendsPaid',
        'netCommonStockIssuance',
        'netPreferredStockIssuance',
        'repurchaseOfCapitalStock',
        'netOtherFinancingCharges',
        'cashFlowFinancingActivities',
        'freeCashFlow']

company_cash_median = cash_sheets.groupby(['company','industry'])[cols].median().reset_index()

In [32]:
# Convert to a dictionaries for easy lookup

company_inc_median = df_company_inc_median.set_index('company').T.to_dict()

## Create Dictionaries for Company and Industry


In [ ]:
ticker_list = income_sheets['company'].unique()

company = {}

for ticker in ticker_list:
  print(ticker)
  company[ticker] = {}
  company[ticker]['income'] = income_sheets[income_sheets['company'] == ticker]
  company[ticker]['cash'] = cash_sheets[cash_sheets['company'] == ticker]
  company[ticker]['balance'] = balance_sheets[balance_sheets['company'] == ticker]
  company[ticker]['industry'] = income_sheets[income_sheets['company'] == ticker]['industry'].unique()[0]

### Score Income Sheet

0 - Poor
4 - Excellent

If Revenue is low or zero then disregard.

If gross profit is low or negative then the business is not viable as it currently stands.

If Operating Income is low or negative then general admin costs, marketing, R&D required for the business operation 
renders the business not profitable.

If net income is low or negative then having allowed for all costs including tax and interest, the business is 
not making any money.

If any of the above applies then we need to apply a ceiling, so that at best the rating applied is 2 (hold). 
Taking a mean of scores and applying a ceiling of 3 will result in a rating of 3 where several factors indicate 
a positive trend and also weigh up the relative strength of trends. if the positive trends are weak and offset by 
declining trends in other factors then the net effect is to push rating to 0 or 1 (indicating sell or strong sell).

Where business is making money:
    
Declining sales revenue is a concern.

Declining gross profit is a worrying indicator even if sales are increasing.

Declining operating income indicates that general costs including marketing and overheads are increasing even if 
trends for sales and profit are positive.

Declining net income indicates that effect of tax and interest is affecting the profitability of the business. 

Taking a mean score will work as strong declines in any factor prevent it from getting top score. More than one 
decline will take the score to a hold or below:
    
    4 4 4 -> 4
    2 4 4 -> 3
    1 4 4 -> 3
    2 3 4 -> 3
    2 2 4 -> 2
    

In [ ]:
def scoreIncome(S_incomeTrend, S_Median):

  revenueThreshold = 100000000
  grossProfitThreshold = 1000000

  '''
  Score company on basis of income sheet
  '''

  s1 = {}

  # Total Revenue
    
  # If the revenue is below threshold then assign lowest score of 0. Otherwise, consider trend, distinguishing
  # positive and negative trends and degree of gradient. 

  feature = 'totalRevenue'

  if S_incomeMedian[feature] < revenueThreshold:
    s1[feature] = 0
  else:
    if S_incomeTrend[feature] < -2:
      s1[feature] = 1
    elif S_incomeTrend[feature] < 0:
      s1[feature] = 2
    elif S_incomeTrend[feature] < 2:
      s1[feature] = 3
    else:
      s1[feature] = 4

  # Gross Profit

  # Only consider companies which are making a profit above threshold.Favour ones with positive trend.

  feature = 'grossProfit'

  if S_incomeMedian['grossProfit'] < grossProfitThreshold:
    s1[feature] = 0
  else:
    if S_incomeTrend[feature] < -2:
      s1[feature] = 1
    elif S_incomeTrend[feature] < 0:
      s1[feature] = 2
    elif S_incomeTrend[feature] < 2:
      s1[feature] = 3
    else:
      s1[feature] = 4
    
  # Operating Income

  # Gives us an indication of the effect of general costs including admin, marketing and R&D. Instead of looking
  # at SGA which may be rising in line with growth in business, the trend for operating income is a good indicator
  # of general direction.
    
  # If it is negative then the business is making a loss having taken into consideration of its operating costs.
  # As this brings into question whether the business is viable based on past performance.
    
  feature = 'operatingIncome'

  if S_incomeMedian[feature] < 0:
    s1[feature] = 0
  else:
    if S_incomeTrend[feature] < -2:
      s1[feature] = 1
    elif S_incomeTrend[feature] < 0:
      s1[feature] = 2
    elif S_incomeTrend[feature] < 2:
      s1[feature] = 3
    else:
      s1[feature] = 4

  # Net Income

  feature = 'netIncome'

  if S_incomeMedian[feature] < 0:
    s1[feature] = 0
  else:
    if S_incomeTrend[feature] < -2:
      s1[feature] = 1
    elif S_incomeTrend[feature] < 0:
      s1[feature] = 2
    elif S_incomeTrend[feature] < 2:
      s1[feature] = 3
    else:
      s1[feature] = 4

  # Scoring

  # Determine mean score. If any of the scores are 0 then apply a ceiling of 2.
  
  l_scores = [s1['totalRevenue'],s1['grossProfit'],s1['operatingIncome'],s1['netIncome']]
    
  meanScore = int(np.mean(l_scores))

  if min(l_scores) == 0:
    if meanScore > 2:
      meanScore = 2

  score = {'score': meanScore,
           'scoreBreakdown': s1}

  return score


In [ ]:
def scoreBalance(S_Trend, S_Median):

  equityThreshold = 100000000
  cashThreshold = 10000000
  debtCapitalThreshold = 90
  debtAssetThreshold = 90

  s1 = {}

  # Stockholders Equity
    
  # If stockholders equity is below threshold then treat net asset value as negligible 
  # and score as 0. Otherwise consider trend to determine score.

  feature = 'stockholdersEquity'

  if S_Median[feature] < equityThreshold:
    s1[feature] = 0
  else:
    if S_Trend[feature] < -2:
      s1[feature] = 1
    elif S_Trend[feature] < 0:
      s1[feature] = 2
    elif S_Trend[feature] < 2:
      s1[feature] = 3
    else:
      s1[feature] = 4
    
  # Retained Profits

  # Positive and high retained profits is a sign of strength but a number of factors can influence whether this
  # is a good thing. Negative retained profits is bad as it signals losses in previous years and perhaps dividend
  # payouts not funded from earnings. 
    
  # Consider positive retained profit as ok and apply max score. For negative, set score to 0 and upgrade score 
  # if there is a positive trend.
    
  feature = 'retainedEarnings'

  if S_Median[feature] < 0:
    
    # Set score to 0 and upgrade if trend is positive
    
    s1[feature] = 0
  
    if S_Trend[feature] > 2:
      s1[feature] = 2
    elif S_Trend[feature] > 0:
      s1[feature] = 1
  else:
    # If retained profits is positive, set score to max
    
    s1[feature] = 4

  # Cash Assets

  # One would expect all companies to hold a minimum of cash to cover short term liabilities so any below
  # specified threshold should be scored 0. Any above threshold should be scored according to trend. 

  # A positive trend (>0) is reassuring while a declining trend may raise concerns even if historical cash 
  # positions have been good).

  feature = 'cashEquivalents'

  if S_Median[feature] < cashThreshold:
    s1[feature] = 0
  else:
    if S_Trend[feature] < -2:
      s1[feature] = 1
    elif S_Trend[feature] < 0:
      s1[feature] = 2
    elif S_Trend[feature] < 2:
      s1[feature] = 3
    else:
      s1[feature] = 4
    
  # Working Capital

  # Various factors such as accounts receivables and inventory can affect this figure from one period to another. 
  # Also a positive trend is not necessarily a good thing as it may indicate in increase in inventory and goods 
  # not sold as quickly. 
    
  # So let's concern ourselves only where negative working capital. Where this is the case, a negative trend 
  # indicates a worsening gap between current assets and liabilities while a positive trend indicates an 
  # improvement.

  # A negative working capital depends on industry and with a strong positive trend, should not by itself be a 
  # reason for not buying the stock. If other indicators are week then this will drive the overall score down 
  # anyway.

  feature = 'workingCapital'
    
  if S_Median[feature] < 0:
    if S_Trend[feature] < -2:
      s1[feature] = 0
    elif S_Trend[feature] < 0:
      s1[feature] = 1
    elif S_Trend[feature] < 2:
      s1[feature] = 2
    else:
      s1[feature] = 3
  else:
    # Set score to max if working capital is positive
    s1[feature] = 4
    
  # Quick Ratio

  # Determines how strong the company is in paying off short term liabilities using cash assets. < 1 is bad,
  # > 3 is good.

  feature = 'quickRatio'
    
  if S_Median[feature] < 0.5:
    s1[feature] = 0
  elif S_Median[feature] < 1.0:
    s1[feature] = 1
  elif S_Median[feature] < 2.0:
    s1[feature] = 2
  elif S_Median[feature] < 3.0:
    s1[feature] = 3
  else:
    s1[feature] = 4

  # The median value for ratio may be high for some reason but there may be a steep downward trend indicating
  # a deterioration.
  # Alternatively median quick ratio may be low for some reason but there may be a steep upward trend indiating
  # a rapid improvement.
    
  # Apply adjustment to score considering gradient and direction, while setting a floor of 0 and max of 4.

  if S_Trend[feature] < -2:
    # Steep decline
    s1[feature] = max(s1[feature] - 2,0)
  elif S_Trend[feature] < 0:
    # Gentle decline
    s1[feature] = max(s1[feature] - 1,0)
  elif S_Trend[feature] < 2:
    # Gentle incline
    s1[feature] = min(s1[feature] + 1,4)
  else:
    # Steep incline
    s1[feature] = 4
        
  # Current Ratio

  feature = 'currentRatio'
    
  if S_Median[feature] < 0.5:
    s1[feature] = 0
  elif S_Median[feature] < 1.0:
    s1[feature] = 1
  elif S_Median[feature] < 2.0:
    s1[feature] = 2
  elif S_Median[feature] < 3.0:
    s1[feature] = 3
  else:
    s1[feature] = 4
    
  # The median value for ratio may be high for some reason but there may be a steep downward trend indicating
  # a deterioration.
  # Alternatively median quick ratio may be low for some reason but there may be a steep upward trend indiating
  # a rapid improvement.
    
  # Applying adjustment to score considering gradient and direction, while setting a floor of 0 and max of 4.

  if S_Trend[feature] < -2:
    # Steep decline
    s1[feature] = max(s1[feature] - 2,0)
  elif S_Trend[feature] < 0:
    # Gentle decline
    s1[feature] = max(s1[feature] - 1,0)
  elif S_Trend[feature] < 2:
    # Gentle incline
    s1[feature] = min(s1[feature] + 1,4)
  else:
    # Steep incline
    s1[feature] = 4
        
  # Solvency Rato - Debt to Capital Ratio

  # Indication of level of leverage. Consider industry norm plus max threshold of 75%.
    
  # If stockholder equity is below threshold then set a default score of 0 as it is meaningless to calculate 
  # a ratio where equity is negligible or even negative.
    
  # Set score to 0 if leverage is > 80%

  feature = 'debtCapitalRatio'

  if S_Median['stockholdersEquity'] < equityThreshold:
    s1[feature] = 0
  elif S_Median[feature] > debtCapitalThreshold:
    s1[feature] = 0 
  elif S_Median[feature] > 60:
    s1[feature] = 1
  elif S_Median[feature] > 50:
     s1[feature] = 2
  elif S_Median[feature] > 30:
     s1[feature] = 3
  else:
     s1[feature] = 4
    
  # Check Industry Averages and adjust score upwards if it is better and lower if it is worse
  # Apply a tolerance of 5 and do nothing where ration is in the vicinity of median

  # if S_Median[feature] < (industryRatio[feature]['median'] - 5):
  #    s1[feature] = min(s1[feature] + 1,4)
  # elif S_Median[feature] > (industryRatio[feature]['median'] + 5):
  #   s1[feature] = max(s1[feature] - 1,0)
 
  # Now consider trend
    
  # If the score is not 0 then adjust according the gradient and direction of trend.

  # Reasoning: if the score is high (3 or 4) but trend is upwards (increasing leverage) then downgrade depending
  # on whether the gradient is steep. If the score is low (1 or 2) but the trend is downwards (decreasing leverage)
  # then upgrade depending on whether gradient is steep.
    
  # The balance we are looking to achieve here is the trade off between risk associated with high leverage and 
  # trend which signals improvement or decline.

  if s1[feature] > 0:

    if S_Trend[feature] < -2:
      # Steep decline
      s1[feature] = min(s1[feature] + 1,4)
    elif S_Trend[feature] < 0:
      # Gentle decline
      s1[feature] = min(s1[feature] + 1,4)
    elif S_Trend[feature] < 2:
      # Gentle incline
      s1[feature] = min(s1[feature] - 1,0)
    else:
      # Steep incline
      s1[feature] = min(s1[feature] - 1,0)
    
  # Solvency - Debt to Asset Ratio

  # Given that shareholder equity is negative or small in many cases, it makes sense to consider debt to asset
  # ratio as well to assess levels of debt relative to assets. 

  # Also we want to create a bias against companies with high levels of debt so considering both scores will help
  # this. 

  # If ratio is above threshold then set score to 0.

  feature = 'debtCapitalRatio'

  if S_Median[feature] > debtAssetThreshold:
    s1[feature] = 0 
  elif S_Median[feature] > 60:
    s1[feature] = 1
  elif S_Median[feature] > 50:
     s1[feature] = 2
  elif S_Median[feature] > 30:
     s1[feature] = 3
  else:
     s1[feature] = 4

  # Check Industry Averages and adjust score upwards if it is better and lower if it is worse
  # Apply a tolerance of 5 and do nothing where ration is in the vicinity of median

#  if S_Median[feature] < (industryRatio[feature]['median'] - 5):
#    s1[feature] = min(s1[feature] + 1,4)
#  elif S_Median[feature] > (industryRatio[feature]['median'] + 5):
#    s1[feature] = max(s1[feature] - 1,0)
    
  # Now consider trend
    
  # If the score is not 0 then adjust according the gradient and direction of trend.

  # Reasoning: if the score is high (3 or 4) but trend is upwards (increasing leverage) then downgrade depending
  # on whether the gradient is steep. If the score is low (1 or 2) but the trend is downwards (decreasing leverage)
  # then upgrade depending on whether gradient is steep.
    
  # The balance we are looking to achieve here is the trade off between risk associated with high leverage and 
  # trend which signals improvement or decline.

  if s1[feature] > 0:

    if S_Trend[feature] < -2:
      # Steep decline
      s1[feature] = min(s1[feature] + 1,4)
    elif S_Trend[feature] < 0:
      # Gentle decline
      s1[feature] = min(s1[feature] + 1,4)
    elif S_Trend[feature] < 2:
      # Gentle incline
      s1[feature] = min(s1[feature] - 1,0)
    else:
      # Steep incline
      s1[feature] = min(s1[feature] - 1,0)

  # Scoring

  # Determine mean score. If any of the scores are 0 then apply a ceiling of 2.
  
  l_scores = list(s1.values())
    
  meanScore = int(np.mean(l_scores))

  if min(l_scores) == 0:
    if meanScore > 2:
      meanScore = 2

  score = {'score': meanScore,
           'scoreBreakdown': s1}

  return score

## Score Cash Sheet

0 - Poor
4 - Excellent

Free cashflow should be positive.

Determine what percentage of free cash flow is spent on capital expenditure.

Is stock buy backs and dividends coverned by free cashflow.

How is money spent:
    
    Dividends, stock buy backs acquisitions, investment in own company
    
How is finance raised:
    
    Stock issuance, debt issuance, asset sales
    
May be an idea to just concentrate on free cash flow and related trend, cover for spend on share buy backs,
dividends and capex.
